In [1]:
import pandas as pd
import numpy as np
import random
import os
from tqdm.notebook import tqdm

import geopandas as gpd
from shapely.geometry import Point
import folium

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.options.display.float_format = '{:.5}'.format #小數點後保留5位
pd.options.display.max_rows = None

%matplotlib inline
import warnings
warnings.filterwarnings('ignore') # You can ignore the Shapely GEOS warning :-)

In [2]:
!pip install haversine

In [4]:
DATA_PATH = '/content'
#Load files
train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
samplesubmission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

#Preview train dataset
train.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.00010834,0.60302,-6.5331e-05,0.25567,-98.594,...,3664.4,6.1086e+04,2615.1,15.569,0.27229,-12.629,35.632,-138.79,30.752,3.751
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,2.0527e-05,0.72821,1.361e-05,0.13099,16.593,...,3651.2,6.6969e+04,3174.6,8.6906,0.25683,30.359,39.558,-145.18,27.252,4.0252
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.00051414,0.7482,0.00038468,0.11002,72.796,...,4217.0,6.0069e+04,3516.3,21.103,0.2511,15.378,30.402,-142.52,26.193,4.2314
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.5,5.1065e+04,4181.0,15.387,0.26204,-11.293,24.38,-132.67,28.829,4.3053
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-7.8766e-05,0.6763,-4.7624e-05,0.12116,4.1213,...,3980.6,6.3751e+04,3355.7,8.1147,0.23585,38.532,37.393,-141.51,22.205,4.3473


In [5]:
train = train[["latitude", "longitude", "year", "week_no", "emission"]].fillna(0)
test = test[["ID_LAT_LON_YEAR_WEEK","latitude", "longitude", "year", "week_no",]].fillna(0) #, "lazy_pred"]]

In [6]:
#https://www.kaggle.com/code/lucasboesen/simple-catboost-6-features-cv-21-7
from sklearn.cluster import KMeans
import haversine as hs

km_train = train.groupby(by=['latitude', 'longitude'], as_index=False)['emission'].mean()
model = KMeans(n_clusters = 7, random_state = 42)
model.fit(km_train)
yhat_train = model.predict(km_train)
km_train['kmeans_group'] = yhat_train

""" Own Groups """
# Some locations have emission == 0
km_train['is_zero'] = km_train['emission'].apply(lambda x: 'no_emission_recorded' if x==0 else 'emission_recorded')

# Distance to the highest emission location
max_lat_lon_emission = km_train.loc[km_train['emission']==km_train['emission'].max(), ['latitude', 'longitude']]
km_train['distance_to_max_emission'] = km_train.apply(lambda x: hs.haversine((x['latitude'], x['longitude']), (max_lat_lon_emission['latitude'].values[0], max_lat_lon_emission['longitude'].values[0])), axis=1)

train = train.merge(km_train[['latitude', 'longitude', 'kmeans_group', 'distance_to_max_emission']], on=['latitude', 'longitude'])
test = test.merge(km_train[['latitude', 'longitude', 'kmeans_group', 'distance_to_max_emission']], on=['latitude', 'longitude'])
#train = train.drop(columns = ["latitude", "longitude"])
#test = test.drop(columns = ["latitude", "longitude"])

In [7]:
train.head()

,latitude,longitude,year,week_no,emission,kmeans_group,distance_to_max_emission
0,-0.51,29.29,2019,0,3.751,0,207.85
1,-0.51,29.29,2019,1,4.0252,0,207.85
2,-0.51,29.29,2019,2,4.2314,0,207.85
3,-0.51,29.29,2019,3,4.3053,0,207.85
4,-0.51,29.29,2019,4,4.3473,0,207.85


In [8]:
train["kmeans_group"].value_counts()

0    31482
6    20193
4    15900
1     8427
5     2703
3      159
2      159
Name: kmeans_group, dtype: int64

In [9]:
test.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,kmeans_group,distance_to_max_emission
0,ID_-0.510_29.290_2022_00,-0.51,29.29,2022,0,0,207.85
1,ID_-0.510_29.290_2022_01,-0.51,29.29,2022,1,0,207.85
2,ID_-0.510_29.290_2022_02,-0.51,29.29,2022,2,0,207.85
3,ID_-0.510_29.290_2022_03,-0.51,29.29,2022,3,0,207.85
4,ID_-0.510_29.290_2022_04,-0.51,29.29,2022,4,0,207.85


In [10]:
train.latitude, train.longtitude = round(train.latitude, 2), round(train.longitude, 2)

In [11]:
#首先創造一個unique的經緯度，就是train再額外加一個欄叫做'location'.
train['location'] = [str(x) + '_' + str(y) for x, y in zip(train.latitude, train.longitude)]

In [12]:
train.head()

,latitude,longitude,year,week_no,emission,kmeans_group,distance_to_max_emission,location
0,-0.51,29.29,2019,0,3.751,0,207.85,-0.51_29.29
1,-0.51,29.29,2019,1,4.0252,0,207.85,-0.51_29.29
2,-0.51,29.29,2019,2,4.2314,0,207.85,-0.51_29.29
3,-0.51,29.29,2019,3,4.3053,0,207.85,-0.51_29.29
4,-0.51,29.29,2019,4,4.3473,0,207.85,-0.51_29.29


In [13]:
train.drop(columns = ["emission"]).head() # X

,latitude,longitude,year,week_no,kmeans_group,distance_to_max_emission,location
0,-0.51,29.29,2019,0,0,207.85,-0.51_29.29
1,-0.51,29.29,2019,1,0,207.85,-0.51_29.29
2,-0.51,29.29,2019,2,0,207.85,-0.51_29.29
3,-0.51,29.29,2019,3,0,207.85,-0.51_29.29
4,-0.51,29.29,2019,4,0,207.85,-0.51_29.29


In [14]:
train["emission"].head() #y

0    3.751
1   4.0252
2   4.2314
3   4.3053
4   4.3473
Name: emission, dtype: float64

In [15]:
test['location'] = [str(x) + '_' + str(y) for x, y in zip(test.latitude, test.longitude)]

In [16]:
test.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,kmeans_group,distance_to_max_emission,location
0,ID_-0.510_29.290_2022_00,-0.51,29.29,2022,0,0,207.85,-0.51_29.29
1,ID_-0.510_29.290_2022_01,-0.51,29.29,2022,1,0,207.85,-0.51_29.29
2,ID_-0.510_29.290_2022_02,-0.51,29.29,2022,2,0,207.85,-0.51_29.29
3,ID_-0.510_29.290_2022_03,-0.51,29.29,2022,3,0,207.85,-0.51_29.29
4,ID_-0.510_29.290_2022_04,-0.51,29.29,2022,4,0,207.85,-0.51_29.29


In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format - line style when plotting the graph
      label - tag for the line
      start - first time step to plot
      end - last time step to plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))

    if type(series) is tuple:

        for series_num in series:
            # Plot the time series data
            plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plot the time series data
      plt.plot(time[start:end], series[start:end], format)

    # Label the x-axis
    plt.xlabel("Time")

    # Label the y-axis
    plt.ylabel("Value")

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

In [19]:
#製作把經緯度考慮進去的dataset
km_class = 1 #0 1 2 3 4 5 6
train_eng = train.loc[train["kmeans_group"] == km_class]
coordinates = zip(train_eng["latitude"],train_eng["longitude"])
coordinates = list(coordinates)
coordinates = set(coordinates)
coordinates = list(coordinates) #497個地點
"""
這裡先只取一兩個，看看學得怎麼樣?
"""
coordinates = coordinates #[:10]

In [20]:
len(train.loc[train["kmeans_group"] == km_class])

8427

In [21]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.3 MB/s eta 0:00:00


In [22]:
from sklearn.utils import shuffle
import catboost as cb

Test = test.drop(columns = ["location","ID_LAT_LON_YEAR_WEEK"]).values
X = train.drop(columns = ["emission","location"]).values
y = train["emission"].values
X, y = shuffle(X, y, random_state=13)
#X = X.astype(np.float32)

offset = int(X.shape[0] * 0.9)

X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

"""
train_dataset = cb.Pool(X_train, y_train)
test_dataset = cb.Pool(X_test, y_test)
"""


'\ntrain_dataset = cb.Pool(X_train, y_train) \ntest_dataset = cb.Pool(X_test, y_test)\n'

In [23]:
train.head()

,latitude,longitude,year,week_no,emission,kmeans_group,distance_to_max_emission,location
0,-0.51,29.29,2019,0,3.751,0,207.85,-0.51_29.29
1,-0.51,29.29,2019,1,4.0252,0,207.85,-0.51_29.29
2,-0.51,29.29,2019,2,4.2314,0,207.85,-0.51_29.29
3,-0.51,29.29,2019,3,4.3053,0,207.85,-0.51_29.29
4,-0.51,29.29,2019,4,4.3473,0,207.85,-0.51_29.29


In [24]:
train.drop(columns = ["emission","location"]).head()

,latitude,longitude,year,week_no,kmeans_group,distance_to_max_emission
0,-0.51,29.29,2019,0,0,207.85
1,-0.51,29.29,2019,1,0,207.85
2,-0.51,29.29,2019,2,0,207.85
3,-0.51,29.29,2019,3,0,207.85
4,-0.51,29.29,2019,4,0,207.85


In [25]:

cat_params = {

    'n_estimators': 799,
    'learning_rate': 0.09180872710592884,
    'depth': 8,
    'l2_leaf_reg': 1.0242996861886846,
    'subsample': 0.38227256755249117,
    'colsample_bylevel': 0.7183481537623551,
    'random_state': 42,
    "silent": True,
    'feature_weights':[1.0,1.0,1.0,1.0,1.0,1.0]
}


In [33]:
param_grid = {
    'n_estimators': [799],  # 将整数值包装在列表中
    'learning_rate': [0.09180872710592884],
    'depth': [8],
    'l2_leaf_reg': [1.0242996861886846],
    'subsample': [0.38227256755249117],
    'colsample_bylevel': [0.7183481537623551],
    'random_state': [42],
    'silent': [True],
    'feature_weights': [[1.0, 1.0, 1.0, 1.0, x, y] for x in range(1,5) for y in range(5)
]
}
#[[1.0, 1.0, 1.0, 1.0, i, 1.0] for i in range(10)]


In [ ]:
pairs = [(x, y) for x in range(1, 11) for y in range(1, 11)]

# 打印结果
for pair in pairs:
    print(pair)

In [36]:
from catboost import CatBoostRegressor

In [37]:
#from catboost import CatBoostRegressor
model = CatBoostRegressor(**cat_params)
# 建立模型
"""
model = CatBoostRegressor(random_state=42,
                         loss_function='RMSE',
                         eval_metric='RMSE',
                         use_best_model=True)
"""

"\nmodel = CatBoostRegressor(random_state=42,\n                         loss_function='RMSE',\n                         eval_metric='RMSE',\n                         use_best_model=True)\n"

In [38]:
# 使用訓練資料訓練模型
model.fit(X_train,y_train, eval_set=(X_test, y_test), verbose=0, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [39]:
from sklearn.model_selection import GridSearchCV
mode1_g = CatBoostRegressor()
grid_search = GridSearchCV(mode1_g, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print("最佳参数:", best_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END colsample_bylevel=0.7183481537623551, depth=8, feature_weights=[1.0, 1.0, 1.0, 1.0, 1, 0], l2_leaf_reg=1.0242996861886846, learning_rate=0.09180872710592884, n_estimators=799, random_state=42, silent=True, subsample=0.38227256755249117; total time=   9.4s
[CV] END colsample_bylevel=0.7183481537623551, depth=8, feature_weights=[1.0, 1.0, 1.0, 1.0, 1, 0], l2_leaf_reg=1.0242996861886846, learning_rate=0.09180872710592884, n_estimators=799, random_state=42, silent=True, subsample=0.38227256755249117; total time=   7.2s
[CV] END colsample_bylevel=0.7183481537623551, depth=8, feature_weights=[1.0, 1.0, 1.0, 1.0, 1, 0], l2_leaf_reg=1.0242996861886846, learning_rate=0.09180872710592884, n_estimators=799, random_state=42, silent=True, subsample=0.38227256755249117; total time=   9.1s
[CV] END colsample_bylevel=0.7183481537623551, depth=8, feature_weights=[1.0, 1.0, 1.0, 1.0, 1, 0], l2_leaf_reg=1.0242996861886846, learning_ra

In [40]:
# 使用最佳参数来训练模型
best_model = CatBoostRegressor(**best_params)
best_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=0, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [41]:
pred_best = best_model.predict(X_test)
print(pred_best)
mse = ((y_test - pred_best)**2).mean(axis=0)
print("mse = ",mse)

[ 16.58506244  20.56502611 142.49837626 ...  77.26454194 185.08938263
  46.59690197]
mse =  104.10891971767873


In [42]:
print(X_test)

[[-3.02000000e+00  3.03840000e+01  2.01900000e+03  6.00000000e+00
   0.00000000e+00  1.47277215e+02]
 [-2.71000000e+00  3.01900000e+01  2.02100000e+03  1.20000000e+01
   0.00000000e+00  1.13691150e+02]
 [-1.21000000e+00  3.03890000e+01  2.02100000e+03  1.10000000e+01
   4.00000000e+00  1.83468354e+02]
 ...
 [-2.72000000e+00  3.02810000e+01  2.01900000e+03  2.20000000e+01
   6.00000000e+00  1.23598786e+02]
 [-1.72000000e+00  3.03820000e+01  2.02000000e+03  3.20000000e+01
   1.00000000e+00  1.48330509e+02]
 [-2.09000000e+00  3.07090000e+01  2.02000000e+03  4.00000000e+00
   6.00000000e+00  1.68275002e+02]]


In [43]:
print(y_test)

[ 13.625375  21.038633 138.13644  ...  81.19991  173.17044   44.521725]


In [44]:
mse = ((y_test - pred)**2).mean(axis=0)
print("mean square error = ", mse)

NameError: ignored

In [45]:
mae = np.sum(np.abs(y_test - pred))/len((y_test - pred))
print("mean absolute error = ", mae)

NameError: ignored

In [109]:
print(pred)

[ 17.54630078  19.56796083 145.24926004 ...  77.769247   187.58857235
  47.9825752 ]


In [46]:
Test_pred = best_model.predict(Test)

In [47]:
print(Test_pred)

[ 2.23473768  3.41302934  3.63257242 ... 28.39793502 28.57845248
 28.43206502]


In [48]:
df_Test_pred = pd.DataFrame(Test_pred)

In [49]:
df_Test_pred[0].shape

(24353,)

In [50]:
# # Create a submission file
sub_file = pd.DataFrame({'ID_LAT_LON_YEAR_WEEK': test.ID_LAT_LON_YEAR_WEEK, 'emission': df_Test_pred[0]})
sub_file.head()

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,2.2347
1,ID_-0.510_29.290_2022_01,3.413
2,ID_-0.510_29.290_2022_02,3.6326
3,ID_-0.510_29.290_2022_03,3.7919
4,ID_-0.510_29.290_2022_04,4.1363


In [51]:
#sub_file.to_csv('sample_data')
sub_file.to_csv('my_data.csv', index=False)

In [52]:
pd.read_csv('my_data.csv')

Output hidden; open in https://colab.research.google.com to view.